In [1]:
import os
import cv2
import numpy as np

In [2]:
open_pic_path = next(os.walk("img/OpenFace"))[2]
open_pic_path = [x for x in open_pic_path if ".jpg" in x]
len(open_pic_path)

1231

In [3]:
close_pic_path = next(os.walk("img/ClosedFace"))[2]
close_pic_path = [x for x in close_pic_path if ".jpg" in x]
len(close_pic_path)

1192

In [4]:
train_open_pic_path = open_pic_path[:1181]
train_close_pic_path = close_pic_path[:1142]
test_open_pic_path = open_pic_path[1181:]
test_close_pic_path = close_pic_path[1142:]

In [5]:
train_picture_path = train_open_pic_path+ train_close_pic_path
test_picture_path = test_open_pic_path + test_close_pic_path

In [6]:
from tensorflow.keras.models import load_model
import torchvision.transforms.functional as TF
import torch

In [7]:
def preprocess_image(image):
    image = TF.to_pil_image(image)
    image = TF.resize(image, (128, 128))
    image = TF.to_tensor(image)
    image = (image - image.min())/(image.max() - image.min())
    image = (2 * image) - 1
    image = torch.transpose(image,0,2).detach().numpy()
    return image

## Open = 1
## Closed = 0

In [8]:
def generate_combine_data(pp_):
    X = []
    Y = []
    for path in pp_:
        if "closed" in path:
            img = cv2.imread(f"img/ClosedFace/{path}")
            Y.append(0)
        else:
            img = cv2.imread(f"img/OpenFace/{path}")    
            Y.append(1)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        preprocessed_image = preprocess_image(img)
        X.append(preprocessed_image)
    return np.array(X),np.array(Y)

In [9]:
train_inputs,train_labels = generate_combine_data(train_picture_path)
test_inputs,test_labels = generate_combine_data(test_picture_path)

In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [11]:
model = InceptionV3(weights=None,input_shape=(128,128,1), classifier_activation="softmax",classes=2)

In [12]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 63, 63, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 63, 63, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 63, 63, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [13]:
model.compile(
              optimizer = 'adam',
              loss = SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'],
             )

In [14]:
model.fit(train_inputs, train_labels, epochs=25,  validation_split = 0.1)

Epoch 1/25
66/66 [==============================] - 18s 136ms/step - loss: 0.8684 - accuracy: 0.6225 - val_loss: 1.0570 - val_accuracy: 0.0000e+00
Epoch 2/25
66/66 [==============================] - 6s 88ms/step - loss: 0.5032 - accuracy: 0.7514 - val_loss: 0.3625 - val_accuracy: 1.0000
Epoch 3/25
66/66 [==============================] - 6s 88ms/step - loss: 0.3195 - accuracy: 0.8729 - val_loss: 0.1976 - val_accuracy: 1.0000
Epoch 4/25
66/66 [==============================] - 6s 91ms/step - loss: 0.1945 - accuracy: 0.9313 - val_loss: 0.4657 - val_accuracy: 0.8627
Epoch 5/25
66/66 [==============================] - 6s 89ms/step - loss: 0.1402 - accuracy: 0.9490 - val_loss: 0.7239 - val_accuracy: 0.6094
Epoch 6/25
66/66 [==============================] - 6s 94ms/step - loss: 0.0946 - accuracy: 0.9683 - val_loss: 0.0270 - val_accuracy: 0.9957
Epoch 7/25
66/66 [==============================] - 6s 92ms/step - loss: 0.0773 - accuracy: 0.9704 - val_loss: 0.1166 - val_accuracy: 0.9528
Epoch 8

In [15]:
def get_confusion(inputs,labels):
    TP = 0
    FN = 0
    TN = 0
    FP = 0
    for inp,label in zip(inputs,labels):
        predict = np.array(model.predict(np.expand_dims(inp,0))[0])
        predict = np.argmax(predict)
        if label ==1 and predict == label:
            TP += 1
        if label ==1 and predict != label:
            FN += 1
        if label ==0 and predict == label:
            TN += 1
        if label ==0 and predict != label:
            FP += 1
    ACC = (TP+TN) / (TP+FP+FN+TN)
    PRECISION = TP / (TP+FP)
    RECALL = TP / (TP+FN)
    F1_SCORE = 2/((1/PRECISION) + (1/RECALL))
    return ACC,PRECISION,RECALL,F1_SCORE

In [16]:
ACC,PRECISION,RECALL,F1_SCORE = get_confusion(train_inputs,train_labels)

In [17]:
print(f"Train ACC --- {ACC}")
print(f"Train PRECISION --- {PRECISION}")
print(f"Train RECALL --- {RECALL}")
print(f"Train F1_SCORE --- {F1_SCORE}")

Train ACC --- 0.9814894532931554
Train PRECISION --- 0.9702479338842975
Train RECALL --- 0.9940728196443692
Train F1_SCORE --- 0.9820158929318277


In [18]:
ACC,PRECISION,RECALL,F1_SCORE = get_confusion(test_inputs,test_labels)

In [19]:
print(f"Test ACC --- {ACC}")
print(f"Test PRECISION --- {PRECISION}")
print(f"Test RECALL --- {RECALL}")
print(f"Test F1_SCORE --- {F1_SCORE}")

Test ACC --- 0.97
Test PRECISION --- 0.9795918367346939
Test RECALL --- 0.96
Test F1_SCORE --- 0.9696969696969697
